In [37]:
import csv
import os
from lxml.etree import Element, SubElement, tostring
from xml.dom.minidom import parseString

In [38]:
def make_voc_dir(env_name:str):
    os.makedirs(env_name+'_voc/Annotations')
    os.makedirs(env_name+'_voc/ImageSets')
    os.makedirs(env_name+'_voc/ImageSets/Main')
    os.makedirs(env_name+'_voc/ImageSets/Layout')
    os.makedirs(env_name+'_voc/ImageSets/Segmentation')
    os.makedirs(env_name+'_voc/JPEGImages')
    os.makedirs(env_name+'_voc/SegmentationClass')
    os.makedirs(env_name+'_voc/SegmentationObject')

In [39]:
def save_xml(image_name, bbox, save_dir='./VOC2007/Annotations', width=1609, height=500, channel=3):
    '''
    将CSV中的一行
    000000001.jpg [[1,2,3,4],...]
    转化成
    000000001.xml

    :param image_name:图片名
    :param bbox:对应的bbox
    :param save_dir:
    :param width:这个是图片的宽度，博主使用的数据集是固定的大小的，所以设置默认
    :param height:这个是图片的高度，博主使用的数据集是固定的大小的，所以设置默认
    :param channel:这个是图片的通道，博主使用的数据集是固定的大小的，所以设置默认
    :return:
    '''
    node_root = Element('annotation')

    node_folder = SubElement(node_root, 'folder')
    node_folder.text = 'JPEGImages'

    node_filename = SubElement(node_root, 'filename')
    node_filename.text = image_name

    node_size = SubElement(node_root, 'size')
    node_width = SubElement(node_size, 'width')
    node_width.text = '%s' % width

    node_height = SubElement(node_size, 'height')
    node_height.text = '%s' % height

    node_depth = SubElement(node_size, 'depth')
    node_depth.text = '%s' % channel

    for x, y, w, h in bbox:
        left, top, right, bottom = max(x, 0), max(y, 0), min(width, x + w), min(height, y + h)
        node_object = SubElement(node_root, 'object')
        node_name = SubElement(node_object, 'name')
        node_name.text = 'SUIT'
        node_difficult = SubElement(node_object, 'difficult')
        node_difficult.text = '0'
        node_bndbox = SubElement(node_object, 'bndbox')
        node_xmin = SubElement(node_bndbox, 'xmin')
        node_xmin.text = '%s' % left
        node_ymin = SubElement(node_bndbox, 'ymin')
        node_ymin.text = '%s' % top
        node_xmax = SubElement(node_bndbox, 'xmax')
        node_xmax.text = '%s' % right
        node_ymax = SubElement(node_bndbox, 'ymax')
        node_ymax.text = '%s' % bottom

    xml = tostring(node_root, pretty_print=True)
    dom = parseString(xml)

    img_name = image_name.split('.')[0]
    save_xml = os.path.join(save_dir, img_name+'.xml')
    with open(save_xml, 'wb') as f:
        f.write(xml)

In [40]:
import cv2
import numpy as np
import pandas as pd
def change2xml(env_name:str):
    # df = pd.read_csv(env_name+'_orig/images/'+mode+'.csv')
    cnt = 0
    path_to_txt = os.path.join(env_name, 'labels')
    path_to_img = os.path.join(env_name, 'images')
    img_files = os.listdir(path_to_img)
    img_files.sort()
    f_trainval = open(env_name+'_voc/ImageSets/Main/trainval.txt', 'a')
    for img_file in img_files:
        img_name = img_file.split('.')[0]
        text_file = img_name+'.txt'

        if not os.path.exists(os.path.join(path_to_txt, text_file)):
            continue
        with open(os.path.join(path_to_txt, text_file)) as f:
            line = f.readline()

        line = line.split(' ')
        img = cv2.imread(os.path.join(path_to_img, img_file))
        x, y, w, h = float(line[1]), float(line[2]), float(line[3]), float(line[4])
        img_width, img_height = img.shape[1], img.shape[0]
        x, y, w, h = int(x*img_width), int(y*img_height), int(w*img_width), int(h*img_height)
        x -= int(w/2)
        y -= int(h/2)
        save_xml(image_name=img_file, bbox=[(x, y, w, h)], save_dir=env_name+'_voc/Annotations', width=img_width, height=img_height)
        f_trainval.write(str(img_name)+'\n')
        # mode = 'train' if np.random.random(1)[0] > 0.2 else 'val'
        mode = 'train' if cnt < 350 else 'val'
        f_mode = open(env_name+'_voc/ImageSets/Main/'+mode+'.txt', 'a')
        f_mode.write(str(img_name)+'\n')
        cnt += 1


In [41]:
# envs = ['HUA', 'PAL_2020', 'PAL_2021', 'RAN', 'TAK', 'LL']
# envs = ['PAL_2020']
# envs = ['LL']
# envs = ['MOO', 'TAH', 'PAL_2021', 'TAK', 'HUA']
# modes = ['train', 'val']
envs = ['PAL']
for env in envs:
    make_voc_dir(env)
    change2xml(env)